In [1]:
from Functions import *
from Optimizations import *
from utils import *
from Saving import *

%load_ext autoreload

%autoreload 2

In [2]:
import time
import yaml

In [3]:
f = flat_sharp_gaussian
grad_f = grad_flat_sharp_gaussian

x_range = [-25, 25]

In [4]:
A = np.array([[1]])
b = np.array([0])
f = QuadraticFunctionInit(A, b)
grad_f = GradQuadraticFunctionInit(A)

x_range = [-15, 15]

In [5]:
num_particles = 1000
process = {}

process["potential_function"] = {"name": "gaussian", "params": g_params}

process["total_iter"] = 100
process["tau"] = 30
process["x_range"] = [-20, 20]

process["start"] = [[np.random.uniform(process["x_range"][0], process["x_range"][1])] for _ in range(num_particles)]
process["gamma"] = 2.5
process["temperature"] = 1
process["epsilon"] = 0

process["weight_function"] = {"name": "none", "params": {"gamma" : 0.999}} 
process["resample_function"] = {"name": "softmax", "params": {"beta": -1.5}} 

process["domain_enforcer"] = {"name": "hyper_cube_enforcer", "params": {"strength": 0.2}}

all_paths = diffusion_resampling(process)


NameError: name 'g_params' is not defined

In [102]:
# want to get order: big_timestep, tau_timestep, particle, point

f, grad_f = get_potential(process)

X = np.linspace(x_range[0], x_range[1], 200)
inp = np.array([X])
Y = f(inp)

inp = all_paths.reshape([np.product(all_paths.shape[:3]), all_paths.shape[-1]]).T
out = f(inp)
all_paths_proc = np.concatenate([inp.T, out.reshape([len(out), 1])], axis=1).reshape(all_paths.shape[0], all_paths.shape[1], all_paths.shape[2], 2)

In [103]:
# only feynman kac densities 
fk_all_paths_proc = all_paths_proc[:, :, -1, :]
fk_all_paths_proc = fk_all_paths_proc.reshape([fk_all_paths_proc.shape[0], fk_all_paths_proc.shape[1], 1, fk_all_paths_proc.shape[2]])

K = multi_gaussian(np.array([[0.6]]))


saved_path = create_animation_1d_pictures_particles(fk_all_paths_proc, X, Y, graph_details={"p_size": 3, #"density_function": None})
                                                                                      "density_function": 
                                                                                      lambda x, p: V(np.array([x]), K, p)})
    
create_animation(saved_path, "fk.mp4", framerate=10)

In [104]:
remove_png(saved_path)
save_config(saved_path, process)

### Things to note:

The beta value of the softmax turns out to be very important. Adjusting that value determines how much to value each respective shallow/flat regions. Seems like changing it can give you a nice stationary distribution around certain falt minima. 

Also the tau value is important. Letting the run run for too long will cause the particles to approach more or less the stationary distribution if ran with soley diffusion. 

Maybe penalize where you started from. 



In [27]:
# full process densities 
K = multi_gaussian(np.array([[0.6]]))


saved_path = create_animation_1d_pictures_particles(all_paths_proc, X, Y, graph_details={"p_size": 3, #"density_function": None})
                                                                                      "density_function": 
                                                                                      lambda x, p: V(np.array([x]), K, p)})
    
create_animation(saved_path, "test.mp4", framerate=15)

In [ ]:
def load_config(load_dir):
    with open(load_path + "process.yml", "r") as f:
        c = yaml.load(f)
    return c